# **Advanced Momentum Trading System**

1. System Overview

Purpose: Identify stocks with strong positive momentum likely to continue in the near term
Complement to Mean Reversion: Focuses on trending markets rather than reversals
Output: Daily list of top stock candidates to buy with confidence scores

2. Data Collection Module

Historical price and volume data using Alpaca API
Market-wide metrics for baseline comparison
Sector/industry classification data
Options data for sentiment analysis (if available)

3. Feature Engineering Module

Multi-timeframe momentum indicators (short, medium, long-term)
Volume confirmation metrics
Volatility normalization
Relative strength vs sector and market
Moving average relationships
Higher-order technical patterns

4. Quality Check Filters

Liquidity threshold filter
Volatility risk assessment
News sentiment screening
Earnings announcement avoidance
Correlation with broader market check
Recent gap analysis

5. Machine Learning Scoring System

Ensemble model approach (Random Forest + XGBoost)
Feature importance tracking
Probability of continuation scoring
Confidence intervals for predictions

6. Position Sizing Recommendations

Volatility-adjusted position sizing
Correlation-based portfolio allocation



In [1]:
# Install required packages for Google Colab
# Comment these out if you're not using Colab or have already installed
import sys
!pip install alpaca-py scikit-learn
!pip install alpaca-trade-api pandas numpy

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class SimpleMomentumSystem:
    def __init__(self, min_price=10.0, min_volume=500000, top_picks=10):
        self.min_price = min_price
        self.min_volume = min_volume
        self.top_picks = top_picks
        self.lookback_days = 100  # Reduced lookback period

    def get_sp500_symbols(self):
        """Get S&P 500 symbols as trading universe"""
        try:
            sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
            return sp500['Symbol'].tolist()
        except:
            # Fallback to common liquid stocks
            return ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'AMD', 'TSLA', 'JPM',
                    'BAC', 'WMT', 'PG', 'JNJ', 'V', 'MA', 'DIS', 'NFLX', 'CSCO', 'ADBE',
                    'CRM', 'XOM', 'CVX', 'PFE', 'MRK', 'KO', 'PEP', 'HD', 'UNH', 'VZ',
                    'T', 'CMCSA', 'INTC', 'ABT', 'COST', 'MCD']

    def calculate_momentum_metrics(self, symbol):
        """Calculate momentum metrics for a given stock"""
        try:
            # Get historical data
            end_date = datetime.now()
            start_date = end_date - timedelta(days=self.lookback_days)
            data = yf.download(symbol, start=start_date, end=end_date, progress=False)

            if len(data) < 50:  # Need sufficient history
                return None

            # Calculate basic momentum metrics
            close = data['Close'].iloc[-1]
            volume = data['Volume'].iloc[-1]

            # Check minimum criteria
            if close < self.min_price or volume < self.min_volume:
                return None

            # Calculate moving averages
            sma_20 = data['Close'].rolling(window=20).mean().iloc[-1]
            sma_50 = data['Close'].rolling(window=50).mean().iloc[-1]

            # Calculate returns over different periods
            returns_5d = data['Close'].pct_change(periods=5).iloc[-1]
            returns_10d = data['Close'].pct_change(periods=10).iloc[-1]
            returns_20d = data['Close'].pct_change(periods=20).iloc[-1]

            # Calculate volume trend
            vol_ratio = data['Volume'].iloc[-5:].mean() / data['Volume'].iloc[-20:].mean()

            # Calculate RSI
            delta = data['Close'].diff()
            gain = delta.where(delta > 0, 0).rolling(window=14).mean()
            loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs)).iloc[-1]

            # Momentum criteria
            above_ma = close > sma_20 and sma_20 > sma_50
            positive_return = returns_5d > 0 and returns_10d > 0
            not_overbought = rsi < 70

            # Only consider stocks that meet all criteria
            if above_ma and positive_return and not_overbought:
                # Calculate composite momentum score
                momentum_score = (
                    returns_5d * 0.4 +
                    returns_10d * 0.3 +
                    returns_20d * 0.2 +
                    (vol_ratio - 1) * 0.1
                )

                # For high-quality candidates, add their metrics to results
                if momentum_score > 0:
                    return {
                        'symbol': symbol,
                        'price': round(close, 2),
                        'momentum_score': round(momentum_score, 3),
                        'above_ma': above_ma,
                        'rsi': round(rsi, 1),
                        'returns_5d': round(returns_5d * 100, 2),  # As percentage
                        'returns_10d': round(returns_10d * 100, 2),  # As percentage
                        'volume_ratio': round(vol_ratio, 2),
                        'price_to_sma20': round(close / sma_20 - 1, 3)  # As percentage above MA
                    }

            return None

        except Exception as e:
            return None

    def find_momentum_stocks(self):
        """Identify stocks with strong momentum characteristics"""
        symbols = self.get_sp500_symbols()
        print(f"Scanning {len(symbols)} stocks for momentum candidates...")

        # Process in smaller batches to avoid overwhelming API
        candidates = []
        batch_size = 10  # Process 10 stocks at a time

        for i in range(0, len(symbols), batch_size):
            batch = symbols[i:min(i+batch_size, len(symbols))]

            for symbol in batch:
                result = self.calculate_momentum_metrics(symbol)
                if result is not None:
                    candidates.append(result)

            # Print progress
            if i % 50 == 0 and i > 0:
                print(f"Processed {i} stocks, found {len(candidates)} candidates so far")

        if candidates:
            # Rank by momentum score and return top picks
            candidates_df = pd.DataFrame(candidates)
            top_picks = candidates_df.sort_values('momentum_score', ascending=False).head(self.top_picks)
            return top_picks
        else:
            print("No momentum candidates found")
            return pd.DataFrame()

def run_momentum_scanner():
    """Run the momentum scanner and return top picks"""
    scanner = SimpleMomentumSystem(min_price=20.0, min_volume=1000000, top_picks=10)
    return scanner.find_momentum_stocks()

# Run the scanner
top_momentum_stocks = run_momentum_scanner()
print("\nTop momentum stocks to buy:")
print(top_momentum_stocks)

Scanning 503 stocks for momentum candidates...
YF.download() has changed argument auto_adjust default to True
Processed 50 stocks, found 0 candidates so far


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-12-15 00:04:18.254793 -> 2025-03-25 00:04:18.254793)')


Processed 100 stocks, found 0 candidates so far
Processed 150 stocks, found 0 candidates so far
Processed 200 stocks, found 0 candidates so far
Processed 250 stocks, found 0 candidates so far
Processed 300 stocks, found 0 candidates so far
Processed 350 stocks, found 0 candidates so far
Processed 400 stocks, found 0 candidates so far
Processed 450 stocks, found 0 candidates so far
Processed 500 stocks, found 0 candidates so far
No momentum candidates found

Top momentum stocks to buy:
Empty DataFrame
Columns: []
Index: []
